# Human Resources Department Attrition Analysis
<span style=color:gray> **In this analysis, we explore the data given to us to help understand the current attrition rate of the company. Certain factors like (Age, Gender, Monthly Income, Over Time Work and a few more) will be looked at to help determine what the main cause of turnover for employees. At the end, we hope to have analyzed the situation and dataset thoroughly and be able to submit the best recommendation to remedy the concern.** </span>

<span style=color:green> <font size="4"> **The code below shows how to load the appropriate libraries where packages can be pulled from and create data.** </span> </font>

In [78]:
shh <- suppressPackageStartupMessages
shh(library(ggplot2))
shh(library(caret))
shh(library(rpart))
shh(library("randomForest"))
install.packages('ggcorrplot')
shh (library(ggcorrplot))
shh (library(pROC))
install.packages('pROC')

# Set standard dimensions for plots.
options(repr.plot.width=4, repr.plot.height=3)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


<span style=color:green> <font size="4"> **This code allows us to import data from a csv file where all data are coming from.** </span> </font>

In [ ]:
hr_data <- read.csv('https://bit.ly/2VTrfPs')

<font size="5"> **The next four data sets you will see allows you to gain basic knowledge about the the Company's Attrition Rate and the variables that dictate it.** </font>

<span style=color:green> <font size="4"> **The code below calculates the dimensions of the dataset.** </span> </font>

In [9]:
dim(hr_data)

[1] 1470   25

<span style=color:green> <font size="4"> **Preview of the first few records in the dataset.** </span> </font>

In [13]:
head(hr_data)

Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,41,Female,Single,Sales,Sales Executive,5993,Yes,Life Sciences,...,3,11,0,0,8,8,6,4,0,5
0,2,49,Male,Married,Research & Development,Research Scientist,5130,No,Life Sciences,...,4,23,1,3,10,1,10,7,1,7
1,4,37,Male,Single,Research & Development,Laboratory Technician,2090,Yes,Other,...,3,15,0,3,7,6,0,0,0,0
0,5,33,Female,Married,Research & Development,Research Scientist,2909,Yes,Life Sciences,...,3,11,0,3,8,1,8,7,3,0
0,7,27,Male,Married,Research & Development,Laboratory Technician,3468,No,Medical,...,3,12,1,3,6,9,2,2,2,2
0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,3,13,0,2,8,0,7,7,3,6


<span style=color:green> <font size="4"> **List and describe each of the fields in the dataset.** </span> </font>

In [14]:
str(hr_data)

'data.frame':	1470 obs. of  25 variables:
 $ Attrition              : int  1 0 1 0 0 0 0 0 0 0 ...
 $ EmployeeNumber         : int  1 2 4 5 7 8 10 11 12 13 ...
 $ Age                    : int  41 49 37 33 27 32 59 30 38 36 ...
 $ Gender                 : Factor w/ 2 levels "Female","Male": 1 2 2 1 2 2 1 2 2 2 ...
 $ MaritalStatus          : Factor w/ 3 levels "Divorced","Married",..: 3 2 3 2 2 3 2 1 3 2 ...
 $ Department             : Factor w/ 3 levels "Human Resources",..: 3 2 2 2 2 2 2 2 2 2 ...
 $ JobRole                : Factor w/ 9 levels "Healthcare Representative",..: 8 7 3 7 3 3 3 3 5 1 ...
 $ MonthlyIncome          : int  5993 5130 2090 2909 3468 3068 2670 2693 9526 5237 ...
 $ OverTime               : Factor w/ 2 levels "No","Yes": 2 1 2 2 1 1 2 1 1 1 ...
 $ EducationField         : Factor w/ 6 levels "Human Resources",..: 2 2 5 2 4 2 4 2 2 4 ...
 $ BusinessTravel         : Factor w/ 3 levels "Non-Travel","Travel_Frequently",..: 3 2 3 2 3 2 3 3 2 3 ...
 $ DistanceFromHome   

<span style=color:green> <font size="4"> **Calculated descriptive stats for each of the fields in the dataset.** </span> </font>

In [15]:
summary(hr_data)

   Attrition      EmployeeNumber        Age           Gender     MaritalStatus
 Min.   :0.0000   Min.   :   1.0   Min.   :18.00   Female:588   Divorced:327  
 1st Qu.:0.0000   1st Qu.: 491.2   1st Qu.:30.00   Male  :882   Married :673  
 Median :0.0000   Median :1020.5   Median :36.00                Single  :470  
 Mean   :0.1612   Mean   :1024.9   Mean   :36.92                              
 3rd Qu.:0.0000   3rd Qu.:1555.8   3rd Qu.:43.00                              
 Max.   :1.0000   Max.   :2068.0   Max.   :60.00                              
                                                                              
                  Department                       JobRole    MonthlyIncome  
 Human Resources       : 63   Sales Executive          :326   Min.   : 1009  
 Research & Development:961   Research Scientist       :292   1st Qu.: 2911  
 Sales                 :446   Laboratory Technician    :259   Median : 4919  
                              Manufacturing Director   :

<span style=color:green> <font size="5"> **As we explore the possible reasons for the cause of Attrition, I  have lister five questions below that can help evaluate that data we have and see which factors cause employees to transfer work.** </span> </font>

<span style=color:red> <font size="4"> **1. Does one gender tend to quit more frequently than the other?** </font> </span>

In [56]:
ggplot(hr_data,aes(x=Gender , fill = factor(Attrition))) + 
    geom_bar(position= "fill") + xlab("Gender") + ylab("Attrition") +
scale_fill_manual(values = c("grey", "red"), name = "Turnover Rate", labels = c("Stayed", "Left"))

<span style=color:green> <font size="4"> The bar chart above depicts the number of turnover the company had within each gender against how much emlpoyees they have.
More male employees seem to churn and transfer work than female ones. </span> </font>

In [36]:
ggplot(hr_data, aes(x=Gender, y=Attrition)) + 
    geom_line() + (aes(color = Gender))

<span style=color:green> <font size="4"> An additional line chart was made to emphasize the situation. </span> </font>

<span style=color:red> <font size="4"> **2. Does age appear to make a difference? Are our younger workers less committed to our company?** </font> </span>

In [66]:
ggplot(hr_data,aes(x=Age, fill = factor(Attrition))) +
    geom_density(position= "fill") + scale_fill_manual(values = c("yellow", "green"), name = "Turnover Rate", labels = c("Stayed", "Left"))

<span style=color:green> <font size="4"> From the density chart above, it looks like that the younger generation (millennials and generation Z) have the tendency to transfer work more than the older generation. </span> </font>

<span style=color:red> <font size="4"> **3. Are we working our employees too hard? Is our overtime requirement for some employees driving them away?** </font> </span>

In [74]:
ggplot(hr_data,aes(OverTime,fill=factor(Attrition))) +
    geom_bar() + xlab("OverTime") + ylab("Attrition") +
scale_fill_manual(values = c("lightblue", "purple"), name = "Turnover Rate", labels = c("Stayed", "Left"))

<span style=color:green> <font size="4"> The bar chart shows you that even though there are fewer people who does overtime for the company, the turnover rate is still higher for them, compared to the people who don't do overtime. </span> </font>

In [73]:
ggplot(hr_data, aes(x=OverTime, y=Attrition)) + 
    geom_line() + (aes(color = OverTime)) +
scale_fill_manual(values = c("lightblue", "purple"), name = "Turnover Rate", labels = c("Stayed", "Left"))

<span style=color:red> <font size="4"> **4. Our concern is that the highest performing employees are leaving at a higher rate. Is that true?** </font> </span>

In [76]:
ggplot(hr_data,aes(Attrition,PerformanceRating)) + geom_jitter()+ ylab("PerformanceRating")+ xlab("Attrition")

<span style=color:green> <font size="4"> According to the scatterplot above, you'll be happy to know that although there is a attrition in the company, we are able to retain people who have a high performance rating for the most part. </span> </font>

<span style=color:red> <font size="4"> **5. Is this about money? Are we not paying employees enough? Were the raises not strong enough? Is that why they leave?** </font> </span>

In [58]:
ggplot(hr_data, aes(x = MonthlyIncome, fill = Attrition)) + 
    geom_dotplot() + (aes(color = Gender))

`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.


<span style=color:green> <font size="4"> You will see from the dotplot above that the employees who receive lower wages tend to leave the company more so than the ones with a higher pay. </span> </font>

<span style=color:red> <font size="5"> **The next data sets are an attempt to predict and split data to come to a more accurate result for the best recommendation.** </span> </font>

<span style=color:green> <font size="4"> **This code below evaluates whether or not there are missing/null values and addresses those, if they exist.** </span> </font>

In [19]:
sum(is.na(hr_data))

[1] 0

In [20]:
cor <- cor(hr_data[c(1,3,8,12:25)])

options(repr.plot.width=7, repr.plot.height=5)
ggcorrplot(cor)

<span style=color:green> <font size="4"> **This code splits the data into training and test sets (using a set seed of "3456".** </span> </font>

In [26]:
# Allows to set the random number generator to the same.
set.seed(3456)  
trainIndex <- createDataPartition(hr_data$Attrition, p = .8, list = FALSE, times = 1)
hr_train <- hr_data[ trainIndex,] 
hr_test <- hr_data[-trainIndex,]

<span style=color:green> <font size="4"> **This code uses logistic regression to create your prediction model.** </span> </font>

In [29]:
# Try to build a regression model with all variables.
hr_train_model <- glm (Attrition ~ Age + Gender + MaritalStatus + MonthlyIncome, 
                          data=hr_train, family=binomial)

summary(hr_train_model)


Call:
glm(formula = Attrition ~ Age + Gender + MaritalStatus + MonthlyIncome, 
    family = binomial, data = hr_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1450  -0.6253  -0.4929  -0.2987   2.7028  

Coefficients:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)          -7.442e-01  4.282e-01  -1.738 0.082244 .  
Age                  -3.137e-02  1.098e-02  -2.856 0.004294 ** 
GenderMale            3.057e-01  1.746e-01   1.750 0.080050 .  
MaritalStatusMarried  3.146e-01  2.501e-01   1.258 0.208392    
MaritalStatusSingle   1.029e+00  2.481e-01   4.148 3.35e-05 ***
MonthlyIncome        -9.775e-05  2.768e-05  -3.531 0.000414 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1023.56  on 1175  degrees of freedom
Residual deviance:  942.48  on 1170  degrees of freedom
AIC: 954.48

Number of Fisher Scoring iterations: 5


<span style=color:green> <font size="4"> **This code creates a decision tree to help calculate the prediction model.** </span> </font>

In [106]:
hr_train_model <- rpart(Attrition~ Age + Gender + MonthlyIncome + OverTime + Department,
        method="class", data=hr_data)

print(hr_train_model)
plotcp(hr_train_model)

# create additional plots
par(mfrow=c(1,2)) # two plots on one page

# plot tree
plot(hr_train_model, uniform=TRUE,
   main="Attrition Rate")
text(hr_train_model, use.n=TRUE, all=TRUE, cex=.2)

n= 1470 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 1470 237 0 (0.8387755 0.1612245)  
  2) OverTime=No 1054 110 0 (0.8956357 0.1043643) *
  3) OverTime=Yes 416 127 0 (0.6947115 0.3052885)  
    6) MonthlyIncome>=2475 347  79 0 (0.7723343 0.2276657) *
    7) MonthlyIncome< 2475 69  21 1 (0.3043478 0.6956522) *


<span style=color:green> <font size="4"> **This code assess the importance of each variable.** </span> </font>

In [30]:
varImp(hr_train_model)

,Overall
,<dbl>
Age,2.855730
GenderMale,1.750394
MaritalStatusMarried,1.257999
MaritalStatusSingle,4.148028
MonthlyIncome,3.531139


In [31]:
# Evaluate variable importance
varImp(hr_train_model, scale=FALSE)

importance <- as.data.frame(varImp(hr_train_model))
importance <- data.frame(Importance = immportnace$Overall,
                        Fiel = rownames(importance))
importance[order(-importance$Importance),]


,Overall
,<dbl>
Age,2.855730
GenderMale,1.750394
MaritalStatusMarried,1.257999
MaritalStatusSingle,4.148028
MonthlyIncome,3.531139


ERROR: Error in data.frame(Importance = immportnace$Overall, Fiel = rownames(importance)): object 'immportnace' not found


<span style=color:green> <font size="4"> **This code adds and populates the "Attrition" prediction in the test dataset.** </span> </font>

In [32]:
hr_test$Attrition_prediction <- predict(hr_train_model,newdata=hr_test,type='response')
head(hr_test)

,Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_prediction
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
6,0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,13,0,2,8,0,7,7,3,6,0.3289514
8,0,11,30,Male,Divorced,Research & Development,Laboratory Technician,2693,No,Life Sciences,...,22,1,2,1,1,1,0,0,0,0.1620813
11,0,14,35,Male,Married,Research & Development,Laboratory Technician,2426,No,Medical,...,13,1,5,6,0,5,4,0,3,0.1886208
17,0,21,32,Male,Divorced,Research & Development,Research Scientist,3298,Yes,Life Sciences,...,12,2,5,7,0,6,2,0,5,0.1462027
22,1,27,36,Male,Single,Sales,Sales Representative,3407,No,Life Sciences,...,23,0,4,10,7,5,3,0,3,0.2949338
23,0,28,34,Female,Single,Research & Development,Research Director,11994,No,Life Sciences,...,11,0,4,13,0,12,6,2,11,0.1241285


<span style=color:green> <font size="4"> **This code evaluates the performance of each method/model using the area under the ROC curve.** </span> </font>

In [53]:
ROC <- roc(hr_test$Attrition, hr_test$Attrition_prediction)

plot(ROC)

auc(ROC)

Setting levels: control = 0, case = 1
Setting direction: controls < cases


Area under the curve: 0.6564

<span style=color:green> <font size="4"> **This code writes the "Attrition" predictions to a new file that can be downloaded and used for study and review.** </span> </font>

In [100]:
hr_test_abbreviated <- hr_test[,c(1,2,26)]
head(hr_test_abbreviated)
write.csv(hr_test, "Attrition_test_predictions1.csv", row.names=FALSE)

,Attrition,EmployeeNumber,Attrition_prediction
,<int>,<int>,<dbl>
6,0,8,0.3289514
8,0,11,0.1620813
11,0,14,0.1886208
17,0,21,0.1462027
22,1,27,0.2949338
23,0,28,0.1241285


<span style=color:red> <font size="6"> **RECOMMENDATION FOR MINIMIZING CURRENT RATE OF ATTRITION** </span> </font>

# Attrition Rate Hypothesis

<span style=color:black> After reviewing and analyzing the given data above, factors that contribute to a company's employee decision of leaving are: low-income pay, possible workload overload, a classification for the younger generation and low-performance ratings. Dissatisfaction over job responsibilities, salary or even work-environment can be major variables that can push one out the door. Especially for your millennials who the The Wall Street Journal described as a generation who seeks the "biggest gains in Happiness with Pay." </span>

Link for Article: https://l.facebook.com/l.php?u=https%3A%2F%2Fwww.wsj.com%2Farticles%2Fyounger-workers-report-biggest-gains-in-happiness-with-pay-11567071000%3Ffbclid%3DIwAR1OviZuLCtKBXJ5x3dnlNVTzxtQBj_hPmb2EPhIsJ5316mOunx_sG5DdwA&h=AT0veMbvwpBGMxp_cbaqGtp2dN2dNEvI93p5Jnd2rRApQclJrsgdvT5MWJSVRDopM6QK8d3rXJPx1PHUMmZjC7NRIHVsIpbGK4tNTjItfwxdWz3NI0MgSt2WgzrTBXBsgd-bTg

# Recommendation

 My personal recommendations are to revamp the compensation and benefits package of the company to a more competitive stand point if it can withstand the costs. According to Maslow's Hierarchy of Needs quotes by simplypsychology.org, "Our most basic need is for physical survival, and this will be the first thing that motivates our behavior. Once that level is fulfilled the next level up is what motivates us, and so on." People seek to increase their abilities to provide for their family's basic needs. Another recommendation is to create campaign or activities within the company that can help foster employee engagement and help them be engaged in the workplace and in their duties. This will decrease the possibility of burning out and will increase motivation to do better everyday.